In [ ]:
import pandas as pd

# Define the file paths
file_path_ohs = '/home/dragon/Git/Data/ohs-hc-template_v11.xlsx'
file_path_nte = '/home/dragon/Git/Data/nte-report-41724.xlsx'

# Read the specific sheet from the OHS HC Template file
positions_df = pd.read_excel(file_path_ohs, sheet_name='Positions Data Template', engine='openpyxl')

# Clean up column names by stripping any leading/trailing whitespace or newline characters
positions_df.columns = positions_df.columns.str.strip().str.replace('\n', '')

# Read the NTE REPORT file starting from the second row
nte_df = pd.read_excel(file_path_nte, skiprows=1, engine='openpyxl')

# Create a new DataFrame with specific columns from positions_df
position = positions_df[[
    'OHS PIN', 'FY Position Authorization', 'Supervisor PIN',
     'Division', 'Branch/Program', 'Position Type',
    'Encumbered Position', 'Position Status', 'Employee Status',
    'Employee ID', 'Employee Name', 'Preferred Name', 'Position Title',
    'Position Description Title', 'Pay Plan', 'Minimum Grade',
    'Maximum Grade', 'Career Ladder Position','Hiring Type','Lapse in Appropriations Status',
    'Official Workplace Flexibility (Position)', 'Position Clearance','Position DOE Clearance', 'Notes'
]].copy()



# Add a new column 'Supervisor Role'       
position['Supervisor Role'] = position['OHS PIN'].map(position['Supervisor PIN'].value_counts())

# Fill NaN values with 0
position['Supervisor Role'].fillna(0, inplace=True)

# Rearrange columns so 'Supervisor Role' comes immediately after 'Supervisor PIN'                


# Function to check if Pay Plan is within position grade range
def check_grade_range(row):
    try:
        pay_plan = float(row['Pay Plan'])
        min_grade = float(row['Minimum Grade'])
        max_grade = float(row['Maximum Grade'])
        
        if pay_plan >= min_grade and pay_plan <= max_grade:
            return 'Within Position Grade Range'
        else:
            return 'Outside of Position Grade Range'
    except ValueError:
        return 'Error: Non-numeric value'


# Apply the function to create a new column 'Grade Range Status'
position['Grade Range Status'] = position.apply(check_grade_range, axis=1)

position = position[['OHS PIN', 'FY Position Authorization', 'Supervisor PIN','Supervisor Role',
                     'Division', 'Branch/Program', 'Position Type',
                     'Encumbered Position', 'Position Status', 'Employee Status',
                     'Employee ID', 'Employee Name', 'Preferred Name', 'Position Title',
                     'Position Description Title', 'Pay Plan', 'Minimum Grade',
                     'Maximum Grade', 'Career Ladder Position','Grade Range Status','Hiring Type',
                     'Lapse in Appropriations Status','Official Workplace Flexibility (Position)', 
                     'Position Clearance','Position DOE Clearance', 'Notes'
                     ]]

# Set pandas options to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


# Display the new DataFrame
print("Position DataFrame with rearranged columns and Supervisor Role:")
position


In [ ]:
positions_df.columns

# Chapter 2

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Define the file path for the OHS HC Template
file_path_ohs = '/home/dragon/Git/Data/ohs-hc-template_v11.xlsx'

# Read the 'Vacancy Data' sheet from the Excel file
vacancy_df = pd.read_excel(file_path_ohs, sheet_name='Vacancy Data', engine='openpyxl')

# Extract specific columns from vacancy_df
columns_to_add = [
    'Hire Manager', 'HC Servicing Specialists', 'FedHR Navigation Number', 
    'Nature of Action', 'Current Status', 'Action Owner', 'USA Jobs', 
    '1. PND PRF Submission', '2. PRF Approved', '3. Budget Certification Complete', 
    '4. Recruitment Request Submitted to OCHCO', '5. PD Classification Complete', 
    '6. Recruitment Package Routed to HRMS Staffing POC', '7. Draft Job Analysis Received', 
    '8. Job Analysis Returned', '9. Draft Vacancy Announcement Documents Received', 
    '10. Vacancy Announcement Documents Returned', '11. Vacancy Announcement Open', 
    '12. Vacancy Announcement Closed', '13. Certificate Issued', '14. Certificate Returned', 
    '15. TJO Issued', '16. Security', '17. FJO Issued', '18. EOD Set or Cancellation Date', 
    'EOD Set or Cancellation', 'Certificate Expiration Date','Honorific Title', 'Selectee Legal Last Name',
    'Selectee Legal First Name', 'Suffix', 'Notes'
]

# Select only the required columns from vacancy_df
additional_data = vacancy_df[columns_to_add].copy()

# Concatenate vacancy_df and additional_data horizontally (side by side)
vacancy = pd.concat([vacancy_df, additional_data], axis=1)

# Convert 'Vacant Date' to datetime format
vacancy['Vacant Date'] = pd.to_datetime(vacancy['Vacant Date'], errors='coerce')

# Calculate 'Length of Vacancy (Days)'
vacancy['Length of Vacancy (Days)'] = (datetime.now() - vacancy['Vacant Date']).dt.days

# Extract columns of interest for date calculation
date_columns = vacancy.columns[vacancy.columns.str.startswith('1.') & vacancy.columns.str.endswith('Date')]

# Convert date columns to datetime format
vacancy[date_columns] = vacancy[date_columns].apply(pd.to_datetime, errors='coerce')

# Find the furthest date recorded to the right
vacancy['LastDate'] = vacancy[date_columns].max(axis=1)

# Calculate the number of workdays between 'LastDate' and today's date, excluding weekends
valid_dates = vacancy['LastDate'].notna()
vacancy.loc[valid_dates, 'Days in Stage'] = np.busday_count(vacancy.loc[valid_dates, 'LastDate'].values.astype('datetime64[D]'), np.datetime64('today'))

vacancy = vacancy[[
    'Hire Manager', 'HC Servicing Specialists', 'FedHR Navigation Number', 
    'Nature of Action', 'Current Status', 'Action Owner', 'USA Jobs', 
    '1. PND PRF Submission', '2. PRF Approved', '3. Budget Certification Complete', 
    '4. Recruitment Request Submitted to OCHCO', '5. PD Classification Complete', 
    '6. Recruitment Package Routed to HRMS Staffing POC', '7. Draft Job Analysis Received', 
    '8. Job Analysis Returned', '9. Draft Vacancy Announcement Documents Received', 
    '10. Vacancy Announcement Documents Returned', '11. Vacancy Announcement Open', 
    '12. Vacancy Announcement Closed', '13. Certificate Issued', '14. Certificate Returned', 
    '15. TJO Issued', '16. Security', '17. FJO Issued', '18. EOD Set or Cancellation Date', 
    'EOD Set or Cancellation', 'Certificate Expiration Date','Vacant Date', 'Length of Vacancy (Days)',
    'LastDate','Days in Stage','Honorific Title', 'Selectee Legal Last Name','Selectee Legal First Name', 
    'Suffix', 'Notes'
]]

# Display the updated DataFrame with the new columns
vacancy.head()


# chapter 3

In [1]:
import pandas as pd

# File path
file_path_ohs = '/home/dragon/Git/Data/ohs-hc-template_v11.xlsx'

# Read specific sheets into DataFrames
individual_data_template = pd.read_excel(file_path_ohs, sheet_name='Individual Data Template')
external_detailee_data_template = pd.read_excel(file_path_ohs, sheet_name='External Detailee Data Template')
direct_support_ctr_template = pd.read_excel(file_path_ohs, sheet_name='Direct Support CTR Template')
positions_data_template = pd.read_excel(file_path_ohs, sheet_name='Positions Data Template')

# Extract relevant columns and rename them
individual_data = individual_data_template[['Employee ID', 'Honorific Title', 'Employee Legal Last Name', 'Employee Legal First Name', 'Suffix', 'Preferred Name', 'OHS EOD Date', 'Separation Date', 'Reason for Separation', 'Email Address Username', 'Desk Phone', 'Mobile Phone', 'Office Number']].rename(
    columns={'Employee ID': 'EmployeeID', 'Honorific Title': 'Honorific Title', 'Employee Legal Last Name': 'Employee Legal Last Name', 'Employee Legal First Name': 'Employee Legal First Name', 'Suffix': 'Suffix', 'Preferred Name': 'Preferred Name', 'OHS EOD Date': 'OHS EOD Date', 'Separation Date': 'Separation Date', 'Reason for Separation': 'Reason for Separation', 'Email Address': 'Email Address', 'Desk Phone': 'Desk Phone', 'Mobile Phone': 'Mobile Phone', 'Office Number': 'Office Number'}
)
external_detailee_data = external_detailee_data_template[['Detailee ID', 'Honorific Title', 'Detailee Legal Last Name', 'Detailee Legal First Name', 'Suffix', 'Preferred Name', 'Detail EOD Date', 'Detail End Date', 'Email Address', 'Desk Phone', 'Mobile Phone', 'Office Number']].rename(
    columns={'Detailee ID': 'EmployeeID', 'Honorific Title': 'Honorific Title', 'Detailee Legal Last Name': 'Employee Legal Last Name', 'Detailee Legal First Name': 'Employee Legal First Name', 'Suffix': 'Suffix', 'Preferred Name': 'Preferred Name', 'Detail EOD Date': 'OHS EOD Date', 'Detail End Date': 'Separation Date', 'Email Address': 'Email Address', 'Desk Phone': 'Desk Phone', 'Mobile Phone': 'Mobile Phone', 'Office Number': 'Office Number'}
)
contractor_data = direct_support_ctr_template[['Contractor ID', 'Honorific Title', 'Contractor Legal Last Name', 'Contractor Legal First Name', 'Suffix', 'Preferred Name', 'Onboard Date', 'Offboard Date', 'Email Address',  'GFE Mobile Phone', 'Office Number']].rename(
    columns={'Contractor ID': 'EmployeeID', 'Honorific Title': 'Honorific Title', 'Contractor Legal Last Name': 'Employee Legal Last Name', 'Contractor Legal First Name': 'Employee Legal First Name', 'Suffix': 'Suffix', 'Preferred Name': 'Preferred Name', 'Onboard Date': 'OHS EOD Date', 'Offboard Date': 'Separation Date', 'Email Address': 'Email Address', 'Desk Phone': 'Desk Phone', 'GFE Mobile Phone': 'Mobile Phone', 'Office Number': 'Office Number'}
)

# Add missing columns with NaN values for columns not present in some templates
external_detailee_data['Reason for Separation'] = pd.NA
contractor_data['Reason for Separation'] = pd.NA

# Concatenate these DataFrames into a new DataFrame
employees_data = pd.concat([individual_data, external_detailee_data, contractor_data], ignore_index=True)

# Display the new DataFrame
print("Employees Data with Employee ID, Honorific Title, Employee Legal Last Name, Employee Legal First Name, Suffix, Preferred Name, OHS EOD Date, Separation Date, Reason for Separation, Email Address, Desk Phone, Mobile Phone, and Office Number:")
employees_data

Employees Data with Employee ID, Honorific Title, Employee Legal Last Name, Employee Legal First Name, Suffix, Preferred Name, OHS EOD Date, Separation Date, Reason for Separation, Email Address, Desk Phone, Mobile Phone, and Office Number:


/tmp/ipykernel_16641/4183370429.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  employees_data = pd.concat([individual_data, external_detailee_data, contractor_data], ignore_index=True)


,EmployeeID,Honorific Title,Employee Legal Last Name,Employee Legal First Name,Suffix,Preferred Name,OHS EOD Date,Separation Date,Reason for Separation,Email Address Username,Desk Phone,Mobile Phone,Office Number,Email Address
0,P100001,NaN,Anderson,Karl,NaN,NaN,NaT,NaN,NaN,Karl.Anderson@hq.dhs.gov,202-357-8769,202-309-8569,NaN,NaN
1,P100002,NaN,Backer-Krisel,Stephanie,NaN,NaN,NaT,NaN,NaN,Stephanie.Backer-Krisel@hq.dhs.gov,NaN,202-924-0818,NaN,NaN
2,P100003,NaN,Ballering,Katie,NaN,NaN,NaT,NaN,NaN,Katie.Ballering@hq.dhs.gov,NaN,202-878-0288,NaN,NaN
3,P100004,NaN,Biles,Amber,NaN,NaN,NaT,NaN,NaN,Amber.Biles@hq.dhs.gov,NaN,202-595-4422,NaN,NaN
4,P100005,NaN,Blount,Brett,NaN,NaN,NaT,NaN,NaN,Brett.Blount@hq.dhs.gov,NaN,202-843-2761,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,C100009,NaN,Miranda,Daniel,NaN,NaN,2023-01-23,NaN,NaN,NaN,NaN,NaN,NaN,Daniel.Miranda@associates.hq.dhs.gov
87,C100010,NaN,Robinson,Christopher,Jr.,NaN,NaT,NaN,NaN,NaN,NaN,202-961-4227,NaN,Christopher.Robinsonjr@associates.hq.dhs.gov
88,C100011,NaN,Staats,Kathryn,NaN,NaN,NaT,NaN,NaN,NaN,NaN,202-320-6373,NaN,Kathryn.Staats@associates.hq.dhs.gov
89,C100012,NaN,Tillman,Cole,NaN,NaN,NaT,NaN,NaN,NaN,NaN,202-941-1263,NaN,Cole.Tillman@associates.hq.dhs.gov


In [2]:
import pandas as pd

# File path
file_path_ohs = '/home/dragon/Git/Data/ohs-hc-template_v11.xlsx'

# Read specific sheets into DataFrames
individual_data_template = pd.read_excel(file_path_ohs, sheet_name='Individual Data Template')

# Extract Schedule A column from Individual Data Template
schedule_a_column = individual_data_template['Schedule A']

# Assign Schedule A column to employees_data DataFrame
employees_data['Schedule A'] = schedule_a_column

# Extract "Official Workplace Flexibility (Position)" column from Positions Data Template
wpf_status_column = positions_data_template['Official Workplace Flexibility (Position)']

# Assign "Employee WPF Status" column to employees_data DataFrame
employees_data['Employee WPF Status'] = wpf_status_column

# Extract "Pay Plan" column from External Detailee Data Template
pay_plan_detailee = external_detailee_data_template['Pay Plan']

# Assign "Pay Plan" column to employees_data DataFrame
employees_data['Pay Plan'] = pay_plan_detailee


# Extract "Grade" column from External Detailee Data Template
grade_column = external_detailee_data_template['Grade']

# Extract "Detailee Location State" column from External Detailee Data Template
employee_location_state_column = external_detailee_data_template['Detailee Location State']

# Assign "Grade" and "Employee Location State" columns to employees_data DataFrame
employees_data['Grade'] = grade_column
employees_data['Employee Location State'] = employee_location_state_column

# Define a mapping function to clean the city names
def clean_city(city):
    if city == "National Capital Region":
        return "District of Columbia"
    else:
        return city

# Apply the mapping function to create the "Location City Clean" column
employees_data['Location City Clean'] = external_detailee_data_template['Detailee Location City '].apply(clean_city)



# Extract and rename columns for Security Clearance
security_clearance_individual = individual_data_template['Security Clearance'].rename('Security Clearance')
security_clearance_external = external_detailee_data_template['Security Clearance'].rename('Security Clearance')
security_clearance_ctr = direct_support_ctr_template['Clearance'].rename('Security Clearance')

# Extract and rename columns for DOE Clearance
doe_clearance_individual = individual_data_template['DOE Clearance'].rename('DOE Clearance')
doe_clearance_external = external_detailee_data_template['Security Clearance'].rename('DOE Clearance')
doe_clearance_ctr = direct_support_ctr_template['Clearance'].rename('DOE Clearance')

# Extract Security Reinvestigation Date from Individual Data Template
security_reinvestigation_date = individual_data_template['Security Reinvestigation Date'].rename('Security Reinvestigation Date')

# Add these columns to employees_data DataFrame
employees_data['Security Clearance'] = pd.concat([
    security_clearance_individual, security_clearance_external, security_clearance_ctr
], ignore_index=True)

employees_data['DOE Clearance'] = pd.concat([
    doe_clearance_individual, doe_clearance_external, doe_clearance_ctr
], ignore_index=True)

employees_data['Security Reinvestigation Date'] = security_reinvestigation_date

# Extract columns from External Detailee Template
date_not_to_exceed = external_detailee_data_template['Date Not to Exceed'].rename('Date Not To Exceed')
home_organization = external_detailee_data_template['Home Organization2'].rename('Home Organization')
reimbursable = external_detailee_data_template['Reimbusable'].rename('Reimbursable')

# Extract columns from Direct Support CTR Template
gfe_mobile_phone_id = direct_support_ctr_template['GFE Mobile Phone ID Number'].rename('GFE Mobile Phone ID Number')
gfe_laptop_id = direct_support_ctr_template['GFE Laptop ID Number'].rename('GFE Laptop ID Number')
piv_card = direct_support_ctr_template['PIV Card'].rename('PIV Card')
contract_end_date = direct_support_ctr_template['Contract End Date'].rename('Contract End Date')

# Add these columns to employees_data DataFrame
employees_data['Date Not To Exceed'] = date_not_to_exceed
employees_data['Home Organization'] = home_organization
employees_data['Reimbursable'] = reimbursable
employees_data['GFE Mobile Phone ID Number'] = gfe_mobile_phone_id
employees_data['GFE Laptop ID Number'] = gfe_laptop_id
employees_data['PIV Card'] = piv_card
employees_data['Contract End Date'] = contract_end_date


# Display the updated DataFrame
print("Employees Data with Employee ID, Honorific Title, Employee Legal Last Name, Employee Legal First Name, Suffix, Preferred Name, OHS EOD Date, Separation Date, Reason for Separation, Email Address, Desk Phone, Mobile Phone, Office Number, and Schedule A:")
employees_data


Employees Data with Employee ID, Honorific Title, Employee Legal Last Name, Employee Legal First Name, Suffix, Preferred Name, OHS EOD Date, Separation Date, Reason for Separation, Email Address, Desk Phone, Mobile Phone, Office Number, and Schedule A:


,EmployeeID,Honorific Title,Employee Legal Last Name,Employee Legal First Name,Suffix,Preferred Name,OHS EOD Date,Separation Date,Reason for Separation,Email Address Username,...,Security Clearance,DOE Clearance,Security Reinvestigation Date,Date Not To Exceed,Home Organization,Reimbursable,GFE Mobile Phone ID Number,GFE Laptop ID Number,PIV Card,Contract End Date
0,P100001,NaN,Anderson,Karl,NaN,NaN,NaT,NaN,NaN,Karl.Anderson@hq.dhs.gov,...,Secret,NaN,NaN,NaN,USPHS,NaN,NaN,NaN,NaN,NaN
1,P100002,NaN,Backer-Krisel,Stephanie,NaN,NaN,NaT,NaN,NaN,Stephanie.Backer-Krisel@hq.dhs.gov,...,Secret,NaN,NaN,NaN,USPHS,NaN,NaN,NaN,NaN,NaN
2,P100003,NaN,Ballering,Katie,NaN,NaN,NaT,NaN,NaN,Katie.Ballering@hq.dhs.gov,...,Top Secret,NaN,NaN,NaN,USPHS,NaN,NaN,NaN,NaN,NaN
3,P100004,NaN,Biles,Amber,NaN,NaN,NaT,NaN,NaN,Amber.Biles@hq.dhs.gov,...,TS/SCI,NaN,NaN,NaN,USPHS,NaN,NaN,NaN,NaN,NaN
4,P100005,NaN,Blount,Brett,NaN,NaN,NaT,NaN,NaN,Brett.Blount@hq.dhs.gov,...,Secret,NaN,NaN,NaN,OGC,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,C100009,NaN,Miranda,Daniel,NaN,NaN,2023-01-23,NaN,NaN,NaN,...,Secret,Secret,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,C100010,NaN,Robinson,Christopher,Jr.,NaN,NaT,NaN,NaN,NaN,...,Secret,Secret,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,C100011,NaN,Staats,Kathryn,NaN,NaN,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,C100012,NaN,Tillman,Cole,NaN,NaN,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
import pandas as pd

# File path
file_path_ohs = '/home/dragon/Git/Data/ohs-hc-template_v11.xlsx'

# Read specific sheets into DataFrames
individual_data_template = pd.read_excel(file_path_ohs, sheet_name='Individual Data Template')
internal_detailee_data_template = pd.read_excel(file_path_ohs, sheet_name='Internal Detailee Data Template')

# Merge to find employees currently on internal detail
merged_data = pd.merge(individual_data_template, internal_detailee_data_template, how='inner', on='Employee ID', suffixes=('', '_internal'))

# Create columns based on conditions
employees_data['Internal Detail'] = merged_data.apply(lambda x: 'Yes' if pd.notna(x['Employee ID_internal']) else 'No', axis=1)
employees_data['Internal Detail Location City'] = merged_data['Detail Location City']
employees_data['Internal Detailee Location State'] = merged_data['Detail Location State']
employees_data['Internal Detail Organization'] = merged_data['Detail Organization']
employees_data['Internal Detail EOD Date'] = merged_data['Detaile EOD Date']
employees_data['Internal Detail End Date'] = merged_data['Detail End Date']

# Display the updated DataFrame
print("Employees Data with added internal detail columns:")
print(employees_data.head())


KeyError: 'Employee ID_internal'

In [ ]:
external_detailee_data_template.columns

In [ ]:
individual_data_template.columns

In [ ]:
external_detailee_data_template.columns

In [ ]:
direct_support_ctr_template.columns

In [ ]:
positions_data_template.columns